In [31]:
from crim_intervals import *
import pandas as pd
import ast
import matplotlib
from itertools import tee, combinations
import numpy as np

In [32]:
# THIS IS DEV COPY for use with CLOSE/EXACT Matches

import numpy as np

# Converts lists to tuples

def lists_to_tuples_a(el):
    if isinstance(el, list):
        return tuple(el)
    else:
        return el

# Filters for the length of the Presentation Type in the Classifier

def limit_offset_size(array, limit):
    under_limit = np.cumsum(array) <= limit
    return array[: sum(under_limit)]


# Gets the the list of offset differences for each group 

def get_offset_difference_list_a(group):
    # if we do sort values as part of the func call, then we don't need this first line
    group = group.sort_values("start_offset")
    group["next_offset"] = group.start_offset.shift(-1)
    offset_difference_list = (group.next_offset - group.start_offset).dropna().tolist()
    return offset_difference_list

# The classifications are done here

def classify_offsets_a(offset_difference_list):
    """
    Put logic for classifying an offset list here
    """
    #
    offset_difference_list = limit_offset_size(offset_difference_list, 5000)



    
    alt_list = offset_difference_list[::2]
    
    
    if len(set(offset_difference_list)) == 1 and len(offset_difference_list) > 1:
        return ("PEN", offset_difference_list)
    elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
        return ("ID", offset_difference_list)
    elif len(offset_difference_list) >= 1:
        return ("Fuga", offset_difference_list)
    else: 
        return ("Singleton", offset_difference_list)
    

   
# adds predicted type, offsets and entry numbers to the results

def predict_type_a(group):
    offset_differences = get_offset_difference_list_a(group)
    predicted_type, offsets = classify_offsets_a(offset_differences)

    group["predicted_type"] = [predicted_type for i in range(len(group))]
    group["offset_diffs"] = [offsets for i in range(len(group))]
    group["entry_number"] = [i + 1 for i in range(len(group))]

    return group

In [38]:
df = pd.read_csv(('ave_test_set.csv'), index_col=0)
df = df.drop(columns=["ema", "ema_url", "end_measure", "end_beat"])
df.head()


,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs
0,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,[Superius],1,1.0,0.0,24.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
1,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,[Superius],105,3.0,884.0,910.0,"[4.0, 8.0, 4.0, 4.0, 6.0, 2.0]",28.0
2,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Altus,3,1.0,16.0,40.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
3,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Tenor,5,1.0,32.0,56.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
4,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Bassus,7,1.0,48.0,72.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0


In [143]:
# now process the original match data df by:
# sorting by start offset, then group by pattern generating match and applying the function above for prev entry
# also give each group a number
# sort by group number and start offset

df2 = df.sort_values("start_offset")
df2["group_number"] = df2.groupby('pattern_generating_match').ngroup()
df2 = df2.sort_values(['group_number', 'start_offset'])
df2["prev_entry_off"] = df2["start_offset"].shift(1)
df2["next_entry_off"] = df2["start_offset"].shift(-1)
df2["last_off_diff"] = df2["start_offset"] - df2["prev_entry_off"]
df2["next_off_diff"] = df2["next_entry_off"] - df2["start_offset"]



df2.head()

,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs,group_number,prev_entry_off,next_entry_off,last_off_diff,next_off_diff
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],19,1.0,144.0,156.0,"[3.0, 1.0, 2.0, 2.0, 4.0, 2.0]",14.0,0,NaN,158.0,NaN,14.0
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],20,4.0,158.0,172.0,"[3.0, 1.0, 4.0, 2.0, 4.0, 4.0]",18.0,0,144.0,216.0,14.0,58.0
21,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,Tenor,28,1.0,216.0,225.0,"[3.0, 1.0, 1.0, 1.0, 3.0, 1.0]",10.0,0,158.0,672.0,58.0,456.0
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],85,1.0,672.0,688.0,"[4.0, 2.0, 4.0, 2.0, 4.0, 4.0]",20.0,0,216.0,328.0,456.0,-344.0
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 2.0]",22.0,1,672.0,328.0,-344.0,0.0


In [147]:
# above we need to adapt:  col for PREVIOUS OFFset and NEXT OFFset
# so that we can find PARALLEL (=0) Forward Gaps (the distance to NEXT entry), and Backward Gaps (distance to PREVIOUS Entry)
# All Proximate Matches are part of same sub Group (and go to classifier)
# Parallels are part of Sub Group but NOT part of Classified.  We will filter them OUT before classification
# Forward ONLY gaps are part of NEXT subgroup
# Forward AND Backward Gaps are SINGLETONS:  We will filter then OUT before Classification


df2["parallel"] = df2["last_off_diff"] == 0
df2["forward_gapped"] = df2["next_off_diff"] >= 20
df2["back_gapped"] = df2["last_off_diff"] >= 20
df2["singleton"] = (df2['forward_gapped'] == True) & (df2['back_gapped'] == True)
df2["split_group"] = (df2['forward_gapped'] == True) & (df2['back_gapped'] == False)
df2.head()


    


,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs,group_number,prev_entry_off,next_entry_off,last_off_diff,next_off_diff,parallel,forward_gapped,back_gapped,singleton,split_group
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],19,1.0,144.0,156.0,"[3.0, 1.0, 2.0, 2.0, 4.0, 2.0]",14.0,0,NaN,158.0,NaN,14.0,False,False,False,False,False
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],20,4.0,158.0,172.0,"[3.0, 1.0, 4.0, 2.0, 4.0, 4.0]",18.0,0,144.0,216.0,14.0,58.0,False,True,False,False,True
21,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,Tenor,28,1.0,216.0,225.0,"[3.0, 1.0, 1.0, 1.0, 3.0, 1.0]",10.0,0,158.0,672.0,58.0,456.0,False,True,True,True,False
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],85,1.0,672.0,688.0,"[4.0, 2.0, 4.0, 2.0, 4.0, 4.0]",20.0,0,216.0,328.0,456.0,-344.0,False,False,True,False,False
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 2.0]",22.0,1,672.0,328.0,-344.0,0.0,False,False,False,False,False


In [151]:
# Now we need to interate over each group in order to split the groups at wherever "split_group" is TRUE
# # initalize sub-group counter
sub_counter = 0

# the split-group column tells us if we need to start a new sub-group for that set of matches
# if TRUE, then we need to add a new sub_group_number
# if FALSE, then sub_group is the same
for name, group in df2.groupby("group_number"):
    if df2["split_group"] is False:
        df2["sub_group_id"] = sub_counter   
    else:
        sub_counter = sub_counter + 1
        df2["sub_group_id"] = sub_counter

df2.head(10)



,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs,...,prev_entry_off,next_entry_off,last_off_diff,next_off_diff,parallel,forward_gapped,back_gapped,singleton,split_group,sub_group_id
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],19,1.0,144.0,156.0,"[3.0, 1.0, 2.0, 2.0, 4.0, 2.0]",14.0,...,NaN,158.0,NaN,14.0,False,False,False,False,False,41
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],20,4.0,158.0,172.0,"[3.0, 1.0, 4.0, 2.0, 4.0, 4.0]",18.0,...,144.0,216.0,14.0,58.0,False,True,False,False,True,41
21,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,Tenor,28,1.0,216.0,225.0,"[3.0, 1.0, 1.0, 1.0, 3.0, 1.0]",10.0,...,158.0,672.0,58.0,456.0,False,True,True,True,False,41
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],85,1.0,672.0,688.0,"[4.0, 2.0, 4.0, 2.0, 4.0, 4.0]",20.0,...,216.0,328.0,456.0,-344.0,False,False,True,False,False,41
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 2.0]",22.0,...,672.0,328.0,-344.0,0.0,False,False,False,False,False,41
71,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,[Superius],42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 4.0]",24.0,...,328.0,586.0,0.0,258.0,True,True,False,False,True,41
74,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,74,2.0,586.0,594.0,"[3.0, 1.0, 1.0, 1.0, 2.0, 2.0]",10.0,...,328.0,888.0,258.0,302.0,False,True,True,True,False,41
72,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Altus,106,1.0,888.0,904.0,"[6.0, 2.0, 2.0, 2.0, 4.0, 4.0]",20.0,...,586.0,756.0,302.0,-132.0,False,False,True,False,False,41
177,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Ave Maria,Altus,95,1.0,756.0,776.0,"[6.0, 2.0, 2.0, 2.0, 8.0, 4.0]",24.0,...,888.0,804.0,-132.0,48.0,False,True,False,False,True,41
178,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Ave Maria,Altus,99,1.0,804.0,824.0,"[6.0, 2.0, 2.0, 2.0, 8.0, 4.0]",24.0,...,756.0,864.0,48.0,60.0,False,True,True,True,False,41


In [16]:
classified = df.applymap(lists_to_tuples_a).sort_values("start_offset").groupby("pattern_matched").apply(predict_type_a)
classified["group_number"] = classified.groupby('pattern_generating_match').ngroup()
classified = classified[["group_number", "pattern_generating_match", "pattern_matched", "part", "start_measure", "start_beat", "entry_number", "start_offset", "sum_durs", "offset_diffs", "predicted_type"]]

# remove singleton entries

classified.drop(classified[classified['predicted_type'] == "Singleton"].index, inplace = True)

# now classified results, in order by OFFSET and ENTRY NUMBER, but the group numbers can overlap with each other.
classified.head(25)

classified = classified.sort_values(["group_number", "entry_number"])
classified= classified[["group_number", "entry_number", "pattern_matched", "part", "start_measure", "start_beat", "start_offset", "sum_durs", "offset_diffs", "predicted_type"]]
classified["offset_list_length"] = classified["offset_diffs"].apply(len) + 1
classified.head()


,group_number,entry_number,pattern_matched,part,start_measure,start_beat,start_offset,sum_durs,offset_diffs,predicted_type,offset_list_length
18,0,1,"[-2, -2, -2, -2, -2]",[Superius],19,1.0,144.0,14.0,"[14.0, 58.0, 456.0]",Fuga,4
19,0,2,"[-2, -2, -2, -2, -2]",[Superius],20,4.0,158.0,18.0,"[14.0, 58.0, 456.0]",Fuga,4
21,0,3,"[-2, -2, -2, -2, -2]",Tenor,28,1.0,216.0,10.0,"[14.0, 58.0, 456.0]",Fuga,4
20,0,4,"[-2, -2, -2, -2, -2]",[Superius],85,1.0,672.0,20.0,"[14.0, 58.0, 456.0]",Fuga,4
73,1,1,"[-2, -2, -2, -2, -3]",Tenor,42,1.0,328.0,22.0,"[0.0, 258.0, 302.0]",Fuga,4


In [6]:
classified = classified[classified.entry_number <= classified.offset_list_length]
classified

,group_number,entry_number,pattern_matched,part,start_measure,start_beat,start_offset,sum_durs,offset_diffs,predicted_type,offset_list_length
18,0,1,"[-2, -2, -2, -2, -2]",[Superius],19,1.0,144.0,14.0,"[14.0, 58.0, 456.0]",Fuga,4
19,0,2,"[-2, -2, -2, -2, -2]",[Superius],20,4.0,158.0,18.0,"[14.0, 58.0, 456.0]",Fuga,4
21,0,3,"[-2, -2, -2, -2, -2]",Tenor,28,1.0,216.0,10.0,"[14.0, 58.0, 456.0]",Fuga,4
20,0,4,"[-2, -2, -2, -2, -2]",[Superius],85,1.0,672.0,20.0,"[14.0, 58.0, 456.0]",Fuga,4
73,1,1,"[-2, -2, -2, -2, -3]",Tenor,42,1.0,328.0,22.0,"[0.0, 258.0, 302.0]",Fuga,4
...,...,...,...,...,...,...,...,...,...,...,...
0,40,1,"[4, 1, 2, 2, -3]",[Superius],1,1.0,0.0,32.0,"[16.0, 16.0, 16.0, 836.0]",Fuga,5
2,40,2,"[4, 1, 2, 2, -3]",Altus,3,1.0,16.0,32.0,"[16.0, 16.0, 16.0, 836.0]",Fuga,5
3,40,3,"[4, 1, 2, 2, -3]",Tenor,5,1.0,32.0,32.0,"[16.0, 16.0, 16.0, 836.0]",Fuga,5
4,40,4,"[4, 1, 2, 2, -3]",Bassus,7,1.0,48.0,32.0,"[16.0, 16.0, 16.0, 836.0]",Fuga,5


In [90]:


classify = df.sort_values("start_offset").groupby("pattern_matched")
for pattern_matched, group in classify: 
    classify["prev_ent_offset"] = classify["start_offset"]
    print(prev_ent_offset) 
    # print(group) 

TypeError: 'DataFrameGroupBy' object does not support item assignment

In [141]:
df2["split_group"]

18    False
19     True
21    False
20    False
73    False
      ...  
0     False
2     False
3     False
4      True
1     False
Name: split_group, Length: 219, dtype: bool

In [10]:
#filter out all the parallel entry items
df3 = df2[df2['parallel_entry'] != True] 
df3




pattern_generating_match       pattern_matched  \
pattern_generating_match                                                     
(-2, -2, -2, -2, -2)     18     (-2, -2, -2, -2, -2)  [-2, -2, -2, -2, -2]   
                         19     (-2, -2, -2, -2, -2)  [-2, -2, -2, -2, -2]   
                         21     (-2, -2, -2, -2, -2)  [-2, -2, -2, -2, -2]   
                         20     (-2, -2, -2, -2, -2)  [-2, -2, -2, -2, -2]   
(-2, -2, -2, -2, -3)     73     (-2, -2, -2, -2, -3)  [-2, -2, -2, -2, -3]   
...                                              ...                   ...   
(4, 1, 2, 2, -3)         0          (4, 1, 2, 2, -3)      [4, 1, 2, 2, -3]   
                         2          (4, 1, 2, 2, -3)      [4, 1, 2, 2, -3]   
                         3          (4, 1, 2, 2, -3)      [4, 1, 2, 2, -3]   
                         4          (4, 1, 2, 2, -3)      [4, 1, 2, 2, -3]   
                         1          (4, 1, 2, 2, -3)      [4, 1, 2, 2, -3]   

                            piece_title        part  start_measure  \
pattern_generating_match                                             
(-2, -2, -2, -2, -2)     18   Ave Maria  [Superius]             19   
                         19   Ave Maria  [Superius]             20   
                         21   Ave Maria       Tenor             28   
                         20   Ave Maria  [Superius]             85   
(-2, -2, -2, -2, -3)     73   Ave Maria       Tenor             42   
...                                 ...         ...            ...   
(4, 1, 2, 2, -3)         0    Ave Maria  [Superius]              1   
                         2    Ave Maria       Altus              3   
                         3    Ave Maria       Tenor              5   
                         4    Ave Maria      Bassus              7   
                         1    Ave Maria  [Superius]            105   

                             start_beat  start_offset  end_offset  \
pattern_generating_match                                            
(-2, -2, -2, -2, -2)     18         1.0         144.0       156.0   
                         19         4.0         158.0       172.0   
                         21         1.0         216.0       225.0   
                         20         1.0         672.0       688.0   
(-2, -2, -2, -2, -3)     73         1.0         328.0       348.0   
...                                 ...           ...         ...   
(4, 1, 2, 2, -3)         0          1.0           0.0        24.0   
                         2          1.0          16.0        40.0   
                         3          1.0          32.0        56.0   
                         4          1.0          48.0        72.0   
                         1          3.0         884.0       910.0   

                                             note_durations  sum_durs  \
pattern_generating_match                                                
(-2, -2, -2, -2, -2)     18  [3.0, 1.0, 2.0, 2.0, 4.0, 2.0]      14.0   
                         19  [3.0, 1.0, 4.0, 2.0, 4.0, 4.0]      18.0   
                         21  [3.0, 1.0, 1.0, 1.0, 3.0, 1.0]      10.0   
                         20  [4.0, 2.0, 4.0, 2.0, 4.0, 4.0]      20.0   
(-2, -2, -2, -2, -3)     73  [6.0, 2.0, 4.0, 4.0, 4.0, 2.0]      22.0   
...                                                     ...       ...   
(4, 1, 2, 2, -3)         0   [4.0, 8.0, 4.0, 4.0, 4.0, 8.0]      32.0   
                         2   [4.0, 8.0, 4.0, 4.0, 4.0, 8.0]      32.0   
                         3   [4.0, 8.0, 4.0, 4.0, 4.0, 8.0]      32.0   
                         4   [4.0, 8.0, 4.0, 4.0, 4.0, 8.0]      32.0   
                         1   [4.0, 8.0, 4.0, 4.0, 6.0, 2.0]      28.0   

                             prev_entry_off  parallel_entry  
pattern_generating_match                                     
(-2, -2, -2, -2, -2)     18             NaN           False  
                         19           144.0           False  
      

In [23]:

df3["gapped"] = df3["offset_diff"] >= 20
df3.head()

pattern_generating_match       pattern_matched  \
pattern_generating_match                                                     
(-2, -2, -2, -2, -2)     18     (-2, -2, -2, -2, -2)  [-2, -2, -2, -2, -2]   
                         19     (-2, -2, -2, -2, -2)  [-2, -2, -2, -2, -2]   
                         21     (-2, -2, -2, -2, -2)  [-2, -2, -2, -2, -2]   
                         20     (-2, -2, -2, -2, -2)  [-2, -2, -2, -2, -2]   
(-2, -2, -2, -2, -3)     73     (-2, -2, -2, -2, -3)  [-2, -2, -2, -2, -3]   

                            piece_title        part  start_measure  \
pattern_generating_match                                             
(-2, -2, -2, -2, -2)     18   Ave Maria  [Superius]             19   
                         19   Ave Maria  [Superius]             20   
                         21   Ave Maria       Tenor             28   
                         20   Ave Maria  [Superius]             85   
(-2, -2, -2, -2, -3)     73   Ave Maria       Tenor             42   

                             start_beat  start_offset  end_offset  \
pattern_generating_match                                            
(-2, -2, -2, -2, -2)     18         1.0         144.0       156.0   
                         19         4.0         158.0       172.0   
                         21         1.0         216.0       225.0   
                         20         1.0         672.0       688.0   
(-2, -2, -2, -2, -3)     73         1.0         328.0       348.0   

                                             note_durations  sum_durs  \
pattern_generating_match                                                
(-2, -2, -2, -2, -2)     18  [3.0, 1.0, 2.0, 2.0, 4.0, 2.0]      14.0   
                         19  [3.0, 1.0, 4.0, 2.0, 4.0, 4.0]      18.0   
                         21  [3.0, 1.0, 1.0, 1.0, 3.0, 1.0]      10.0   
                         20  [4.0, 2.0, 4.0, 2.0, 4.0, 4.0]      20.0   
(-2, -2, -2, -2, -3)     73  [6.0, 2.0, 4.0, 4.0, 4.0, 2.0]      22.0   

                             prev_entry_off  parallel_entry  offset_diff  \
pattern_generating_match                                                   
(-2, -2, -2, -2, -2)     18             NaN           False          NaN   
                         19           144.0           False         14.0   
                         21           158.0           False         58.0   
                         20           216.0           False        456.0   
(-2, -2, -2, -2, -3)     73             NaN           False          NaN   

                             gapped  
pattern_generating_match             
(-2, -2, -2, -2, -2)     18   False  
                         19   False  
                         21    True  
                         20    True  
(-2, -2, -2, -2, -3)     73   False

In [92]:
sub_group_id = 1
sub_group = sub_group_id + 1
sub_group

2

In [94]:
gapped = "True"
sub_group_id = 1
if gapped is False:
    df3["sub_group"] = sub_group_id
else:
    df3["sub_group"] = sub_group_id + 1

sub_group


NameError: name 'df3' is not defined

In [12]:
# now run the classifer, but on DF from which the Parallel Entries have been removed!

classified2 = df3.applymap(lists_to_tuples_a).sort_values("start_offset").groupby("pattern_matched").apply(predict_type_a)
classified2["group_number"] = classified2.groupby('pattern_generating_match').ngroup()
classified2 = classified2[["group_number", "pattern_generating_match", "pattern_matched", "part", "start_measure", "start_beat", "entry_number", "start_offset", "prev_entry_off","sum_durs", "offset_diffs", "predicted_type"]]

# remove singleton entries

# classified2.drop(classified[classified['predicted_type'] == "Singleton"].index, inplace = True)

# now classified results, in order by OFFSET and ENTRY NUMBER, but the group numbers can overlap with each other.

classified2 = classified2.sort_values(["group_number", "entry_number"])
classified2= classified2[["group_number", "entry_number", "pattern_matched", "part", "start_measure", "start_beat", "start_offset", "prev_entry_off", "sum_durs", "offset_diffs", "predicted_type"]]
# classified2 = classified2[classified2["predicted_type"] == "PEN"]

classified2.head()

ValueError: 'pattern_generating_match' is both an index level and a column label, which is ambiguous.

In [38]:
l = [16.0, 16.0, 16.0, 836.0]
for count, value in enumerate(l):
    if value >= 5000:
        split_point = [count]
        res = [l[i : j] for i, j in zip([0] + 
          split_point, split_point + [None])] 
  
# printing result 
print ("The split lists are : " +  str(res)) 

    # print(count, value)

The split lists are : [[16.0, 16.0, 16.0], [836.0]]


In [65]:
def group_splitter(classified2):
    od = classified2.offset_diffs
    for count, value in enumerate(od):
        if value >= 5000:
            split_point = [count]
            res = [l[i : j] for i, j in zip([0] + 
                split_point, split_point + [None])] 
  
# printing result 
    print ("The split lists are : " +  str(res)) 
    

    # return str(res))

In [66]:
group_splitter(classified2)

TypeError: '>=' not supported between instances of 'list' and 'int'

In [64]:
print(classified2.offset_diffs)

18          [14.0, 58.0, 456.0]
19          [14.0, 58.0, 456.0]
21          [14.0, 58.0, 456.0]
20          [14.0, 58.0, 456.0]
73               [258.0, 302.0]
                ...            
0     [16.0, 16.0, 16.0, 836.0]
2     [16.0, 16.0, 16.0, 836.0]
3     [16.0, 16.0, 16.0, 836.0]
4     [16.0, 16.0, 16.0, 836.0]
1     [16.0, 16.0, 16.0, 836.0]
Name: offset_diffs, Length: 204, dtype: object


In [86]:
gap = classified2["start_offset"] - classified2["prev_entry_off"]
# return df2

classified2["gapped"] = gap >= 80
c3 = classified2[classified2["gapped"] == True]
c3.head(50)


,group_number,entry_number,pattern_matched,part,start_measure,start_beat,start_offset,prev_entry_off,sum_durs,offset_diffs,predicted_type,gapped
20,0,4,"[-2, -2, -2, -2, -2]",[Superius],85,1.0,672.0,216.0,20.0,"[14.0, 58.0, 456.0]",Fuga,True
74,1,2,"[-2, -2, -2, -2, -3]",Tenor,74,2.0,586.0,328.0,10.0,"[258.0, 302.0]",Fuga,True
72,1,3,"[-2, -2, -2, -2, -3]",Altus,106,1.0,888.0,586.0,20.0,"[258.0, 302.0]",Fuga,True
205,3,3,"[-2, -2, -2, -2, 2]",Altus,69,1.0,544.0,219.0,14.0,"[7.0, 325.0, 120.0, 428.0, 14.0, 28.0]",Fuga,True
206,3,4,"[-2, -2, -2, -2, 2]",Altus,84,1.0,664.0,544.0,14.0,"[7.0, 325.0, 120.0, 428.0, 14.0, 28.0]",Fuga,True
210,3,5,"[-2, -2, -2, -2, 2]",Bassus,129,3.0,1092.0,664.0,16.0,"[7.0, 325.0, 120.0, 428.0, 14.0, 28.0]",Fuga,True
8,5,4,"[-2, -2, -2, 2, -2]",Bassus,139,4.0,1174.0,88.0,18.0,"[16.0, 16.0, 1086.0]",Fuga,True
113,6,2,"[-2, -2, -2, 2, 2]",Altus,27,4.5,215.0,104.0,13.0,"[111.0, 5.0, 178.0, 149.0, 165.0, 384.0, 66.0]",Fuga,True
112,6,4,"[-2, -2, -2, 2, 2]",[Superius],50,4.0,398.0,220.0,11.0,"[111.0, 5.0, 178.0, 149.0, 165.0, 384.0, 66.0]",Fuga,True
114,6,5,"[-2, -2, -2, 2, 2]",Altus,69,2.5,547.0,398.0,14.0,"[111.0, 5.0, 178.0, 149.0, 165.0, 384.0, 66.0]",Fuga,True
